# Westminster Ground Truth Analysis (Google Colab)

This notebook demonstrates the complete workflow for creating orthomosaics from DJI drone imagery and evaluating their accuracy.

**Note**: This notebook is designed for Google Colab. Upload your data to Google Drive and mount it to use this notebook.


## Setup: Install Dependencies and Mount Google Drive


In [ ]:
# Install required packages
!pip install numpy>=1.24.0 opencv-python>=4.8.0 scipy>=1.11.0 scikit-image>=0.21.0
!pip install rasterio>=1.3.0 pillow>=10.0.0 matplotlib>=3.7.0 pandas>=2.0.0
!pip install pyproj>=3.6.0 shapely>=2.0.0 requests>=2.31.0 tqdm>=4.66.0 exifread>=3.0.0 utm>=0.7.0

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Setup complete!")


## Upload Package Files

Upload the `westminster_ground_truth_analysis` package directory to `/content/` or clone from repository.


In [ ]:
# If you have the package in Google Drive, add it to path
# Or clone from repository if available
import sys
from pathlib import Path

# Update this path to point to your package location
package_path = Path("/content/westminster_ground_truth_analysis")
if package_path.exists():
    sys.path.insert(0, str(package_path))
    print(f"Added {package_path} to Python path")
else:
    print("Package not found. Please upload the westminster_ground_truth_analysis directory to /content/")


## Imports and Configuration


In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import package (adjust path as needed)
try:
    from westminster_ground_truth_analysis import (
        GCPParser,
        DJIMetadataParser,
        OrthomosaicPipeline,
        download_basemap,
        compare_orthomosaic_to_basemap,
        visualize_matches,
        visualize_reprojection_errors,
        visualize_camera_poses,
        create_match_quality_report
    )
    print("Package imported successfully!")
except ImportError as e:
    print(f"Import error: {e}")
    print("Please ensure the package is in the Python path")

# Set up paths - UPDATE THESE TO POINT TO YOUR DATA
# Option 1: Data in Google Drive
data_dir = Path("/content/drive/MyDrive/New Westminster Oct _25")
# Option 2: Upload data directly to Colab
# data_dir = Path("/content/data")

output_dir = Path("/content/outputs")
output_dir.mkdir(exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Output directory: {output_dir}")


## 1. Load Ground Control Points


In [ ]:
# Parse GCP file
gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
gcp_parser = GCPParser(str(gcp_file))

gcps = gcp_parser.get_gcps()
print(f"Loaded {len(gcps)} ground control points")

# Display first few GCPs
for gcp in gcps[:5]:
    print(f"  {gcp.name}: X={gcp.x:.2f}, Y={gcp.y:.2f}, Z={gcp.z:.2f}")

# Get bounds
min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
print(f"\nGCP Bounds: X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")


## 2. Process First Dataset (DJI_202510060955_017_25-3288)

**Note**: This may take a while depending on the number of images. Consider processing a subset first for testing.


In [ ]:
# Setup for first dataset
dataset1_dir = data_dir / "DJI_202510060955_017_25-3288"

# Try to parse DJI metadata
dji_metadata1 = DJIMetadataParser(str(dataset1_dir))

# Create pipeline without GCPs first
pipeline1_no_gcp = OrthomosaicPipeline(
    image_dir=str(dataset1_dir),
    output_dir=str(output_dir / "dataset1_no_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=False,
    dji_metadata=dji_metadata1
)

print("Processing dataset 1 (without GCPs)...")
print("This may take 30-60 minutes depending on the number of images...")
output1_no_gcp = pipeline1_no_gcp.run_full_pipeline(output_name="dataset1_no_gcp")


In [ ]:
# Create pipeline with GCPs
pipeline1_with_gcp = OrthomosaicPipeline(
    image_dir=str(dataset1_dir),
    output_dir=str(output_dir / "dataset1_with_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=True,
    gcp_parser=gcp_parser,
    dji_metadata=dji_metadata1
)

print("Processing dataset 1 (with GCPs)...")
output1_with_gcp = pipeline1_with_gcp.run_full_pipeline(output_name="dataset1_with_gcp")


## 3. Visualize Results and Continue with Remaining Steps

The remaining steps (visualization, dataset 2 processing, basemap comparison) follow the same pattern as the local notebook. See `test_westminster_analysis.ipynb` for the complete workflow.
